In [10]:
#讀取資料
import pandas as pd
import pandas.io.sql as pd_sql
import sqlite3 as sql

#從CSV讀取
df = pd.read_csv('../data/fp_demo3.csv')

#從sqLite讀取
#conn = sql.connect("../data/twse.db")
#df = pd.read_sql_query("select * from demo3;", conn)

df.sort_index(ascending=False).head()

,日期,成交股數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,成交筆數,美元／新台幣,人民幣／新台幣,歐元／美元,美元／日幣,英鎊／美元,澳幣／美元,美元／港幣,美元／人民幣,美元／南非幣,紐幣／美元
40,2017/10/31,"22,582,013","5,497,062,659",243.5,245.0,241.5,243.0,0.0,"8,176",30.170,4.551175,1.16440,113.065,1.32145,0.76790,7.80160,6.62905,14.06155,0.68485
39,2017/10/30,"30,241,990","7,358,619,170",242.0,245.0,241.5,243.0,4.0,"10,609",30.205,4.542892,1.16270,113.700,1.31630,0.76745,7.79985,6.64885,14.08700,0.68515
38,2017/10/27,"18,040,541","4,304,460,160",237.5,240.0,237.5,239.0,3.0,"5,715",30.277,4.550936,1.16310,114.110,1.30750,0.76415,7.80370,6.65290,14.30395,0.68355
37,2017/10/26,"12,959,717","3,069,618,429",237.0,238.0,236.0,236.0,-2.0,"4,947",30.229,4.554422,1.18175,113.760,1.32375,0.76870,7.80165,6.63730,14.19160,0.68740
36,2017/10/25,"17,205,187","4,099,505,436",238.5,239.0,237.5,238.0,0.0,"5,808",30.266,4.557152,1.17670,113.860,1.31320,0.77135,7.80330,6.64145,13.73020,0.68910


In [2]:
#定義準備train data方法
def regData(x):
    #格式化數字
    for i, row in x.iterrows():
        x.loc[i, '成交股數'] = row['成交股數'].replace(',', '')
        x.loc[i, '成交筆數'] = row['成交筆數'].replace(',', '')
    #做normalization(第i天資料/i-1天資料)
    for i, row in X.iterrows():
        if i <= len(X)-2:
            x.loc[i, '成交股數'] = int(x.loc[i+1, '成交股數'])/int(x.loc[i, '成交股數'])
            x.loc[i, '成交筆數'] = int(x.loc[i+1, '成交筆數'])/int(x.loc[i, '成交筆數'])
            x.loc[i, '美元／新台幣'] = x.loc[i+1, '美元／新台幣']/x.loc[i, '美元／新台幣']
    return x.head(30) #不取最後一筆

#定義LinearRegression對應data
def linReg(y):
    y.drop(0, axis=0, inplace=True)
    y = y.reset_index(drop=True)
    return y

#定義LogisticRegression對應data
def logReg(y):
    #將價差=0定義為 1 (漲), 價差<=0定義為 0 (跌)
    y.drop(0, axis=0, inplace=True)
    y = y.reset_index(drop=True)
    for i, row in y.iterrows():
        if y.loc[i, '漲跌價差'] >= 0:
            y.loc[i, '漲跌價差'] = 1
        else:
            y.loc[i, '漲跌價差'] = 0
    return y['漲跌價差']
    
#取得必要欄位，以前30筆作為training data
X = df[['成交股數', '成交筆數', '美元／新台幣']].head(31)
X = regData(X)

#y = linReg(df[['收盤價']].head(31)) #LinearRegression
y = logReg(df[['漲跌價差']].head(31)) #LogisticRegression

#分割資料
from sklearn.model_selection import train_test_split

# Split X and y into X_
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

In [3]:
#建立LinearRegression模型
from sklearn.linear_model import LinearRegression, LogisticRegression

#regression_model = LinearRegression() #LinearRegression
regression_model = LogisticRegression() #LogisticRegression
regression_model.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [4]:
#各data係數
for idx, col_name in enumerate(X_train.columns):
    print("The coefficient for {} is {}".format(col_name, regression_model.coef_[0][idx]))

The coefficient for 成交股數 is 0.6415420555032795
The coefficient for 成交筆數 is 0.17409836326558795
The coefficient for 美元／新台幣 is -0.16997536054927276


In [5]:
#截距
intercept = regression_model.intercept_[0]

print("The intercept for our model is {}".format(intercept))

The intercept for our model is -0.1605205605641356


In [6]:
#使用testing data準確率
predictions = regression_model.predict(X_test)
#印出testing data之預測結果
print(predictions)
#準確率
regression_model.score(X_test, y_test)

[ 1.  1.  1.  1.  1.  0.  1.  1.]


0.75